## Import packages

In [ ]:
import dva_sdhdf_combine_v3
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord,EarthLocation, AltAz, ICRS, Galactic, FK4, FK5

## NOTE: if files containing desired days and frequencies have already been created, skip to "Read in selected daily map data"

## Select range of days, frequencies, and bandwidth to average over:

In [ ]:
days_list = list(range(1,48))
freq_list = [520.0,800.0]
freq_width= 5.0

## Read in the scan IDs and times

In [ ]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)
day_array = []

# Read in the data and store it in arrays:
for day in days_list:
    dir_in = '/media/ordoga/DVA_data/survey_phase1_day'+f"{day:02}"+'/'
    with open(dir_in+'DVAsurvey_phase1_day0'+f"{day:02}"+'.txt') as fp:
        for line in fp:       
            scan_id.append(int(line.split()[0]))
            scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
            scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
            day_array.append(day)
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(day_array[i],f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

## Read in frequency array from one of the scans

In [ ]:
scan0 = f"{scan_id[0]:04}"
dir_in = '/media/ordoga/DVA_data/survey_phase1_day'+f"{days_list[0]:02}"+'/'
file = h5py.File(dir_in+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

## Set up frequencies ranges over which to average

In [ ]:
freq_idx = []

for freq_pick in freq_list:
    freq_idx.append(np.where(abs(freq-freq_pick)<=freq_width/2.)[0])

for i in range(0,len(freq_idx)):
    print(i)
    print(freq[freq_idx[i]])
    print('')
    

## Set up new data arrays combining all scans and calculate medians for correlation products
### Note: will take a LONG time to run! (took over 3 hours for 969 scans at 2 frequencies)

In [ ]:
%%time

t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq_idx)])
LL_set = np.empty([0,len(freq_idx)])
reRL_set = np.empty([0,len(freq_idx)])
imRL_set = np.empty([0,len(freq_idx)])
    
    
# Loop through all the scans in the "scan_num" list:   
for i in range(0,len(scan_id)):
    
    # select the file:
    dir_in = '/media/ordoga/DVA_data/survey_phase1_day'+f"{day_array[i]:02}"+'/'
    file = h5py.File(dir_in+'dva_survey_raw_scan_'+f"{scan_id[i]:04}"+'.h5','r')
    print(f"{i:03}",f"{day_array[i]:02}",f"{scan_id[i]:04}",file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']])
    trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    RR = np.empty([len(dataset['metadata']['utc']),len(freq_idx)])
    LL = np.empty([len(dataset['metadata']['utc']),len(freq_idx)])
    reRL = np.empty([len(dataset['metadata']['utc']),len(freq_idx)])
    imRL = np.empty([len(dataset['metadata']['utc']),len(freq_idx)])
    
    for i in range(0,len(freq_idx)):
        #print(i)
        #print(freq[freq_idx[i]])
        RR[:,i] = np.nanmedian(dataset['data'][:,0,freq_idx[i]],axis=1)
        LL[:,i] = np.nanmedian(dataset['data'][:,1,freq_idx[i]],axis=1)
        reRL[:,i] = np.nanmedian(dataset['data'][:,2,freq_idx[i]],axis=1)
        imRL[:,i] = np.nanmedian(dataset['data'][:,3,freq_idx[i]],axis=1)
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,RR],axis=0)
    LL_set = np.concatenate([LL_set,LL],axis=0)
    reRL_set = np.concatenate([reRL_set,reRL],axis=0)
    imRL_set = np.concatenate([imRL_set,imRL],axis=0)
    
t_set_plt = Time(t_set, format='isot',scale='utc').mjd

## Get rid of points where RA=24 artifact occurs

In [ ]:
ra_set_trim = ra_set.copy()
dec_set_trim = dec_set.copy()
az_set_trim = az_set.copy()
el_set_trim = el_set.copy()
t_set_trim = t_set.copy()
t_set_plt_trim = t_set_plt.copy()
noise_set_trim = noise_set.copy()
RR_set_trim = RR_set.copy()
LL_set_trim = LL_set.copy()
reRL_set_trim = reRL_set.copy()
imRL_set_trim = imRL_set.copy()

idxtrim = np.where(trim_flag == 1)[0]
#print(idxtrim)

ra_set_trim[idxtrim] = np.nan
dec_set_trim[idxtrim] = np.nan 
az_set_trim[idxtrim] = np.nan 
el_set_trim[idxtrim] = np.nan 
t_set_trim[idxtrim] = np.nan 
t_set_plt_trim[idxtrim] = np.nan 
noise_set_trim[idxtrim] = np.nan 
RR_set_trim[idxtrim,:] = np.nan 
LL_set_trim[idxtrim,:] = np.nan 
reRL_set_trim[idxtrim,:] = np.nan 
imRL_set_trim[idxtrim,:] = np.nan

## Write out all new arrays and parameters to .npy file

In [ ]:
filename = '../DVA2/DATA/Daily_maps/survey_phase1_upto_day'+str(days_list[-1])+'.npy'
np.save(filename,([RR_set_trim,LL_set_trim,reRL_set_trim,imRL_set_trim,
                    ra_set_trim,dec_set_trim,az_set_trim,el_set_trim,
                    t_set_trim,noise_set_trim,t_set_plt_trim,
                    day_array,scan_id,scan_start,scan_stop,freq_list,freq_width]))


## Map-making code starts here:

## Read in selected daily map data

In [ ]:
data = np.load('../DVA2/DATA/Daily_maps/survey_phase1_upto_day47.npy',allow_pickle=True)

RR_set = data[0].astype(float)
LL_set = data[1].astype(float)
reRL_set = data[2].astype(float)
imRL_set = data[3].astype(float)
ra_set = data[4].astype(float)
dec_set = data[5].astype(float)
az_set = data[6].astype(float)
el_set = data[7].astype(float)
t_set = data[8].copy()
noise_set = data[9].astype(float)
t_set_plt = data[10].astype(float)
day_array = data[11]
scan_id = data[12]
scan_start = data[13]
scan_stop = data[14]
freq_list = data[15]
ra_set_deg = ra_set*360./24.



## Set points when noise source is on to NaN

In [ ]:
wbad = np.where(noise_set == 1.0)
RR_set[wbad] = np.nan
LL_set[wbad] = np.nan
reRL_set[wbad] = np.nan
imRL_set[wbad] = np.nan

## Median subtract reRL and imRL (estimate instrumental pol.) and calculate polarized intensity

In [ ]:
reRL_set_instr = reRL_set.copy()
imRL_set_instr = imRL_set.copy()

for i in range(0,len(freq_list)):
    reRL_med = np.nanmedian(reRL_set[:,i])
    imRL_med = np.nanmedian(imRL_set[:,i])
    reRL_set_instr[:,i] = reRL_set_instr[:,i] - reRL_med
    imRL_set_instr[:,i] = imRL_set_instr[:,i] - imRL_med

PI = np.sqrt(reRL_set_instr**2 + imRL_set_instr**2)
PI[wbad] = np.nan

## Calculate Galactic coordinates (takes several minutes)

In [ ]:
%%time
loc = EarthLocation(lat = 49.3*u.deg, lon = -119.6*u.deg, height = 546*u.m)

wgood = np.where(np.isfinite(el_set))

AltAzcoord = SkyCoord(alt=el_set[wgood]*u.deg, az=az_set[wgood]*u.deg, 
                      obstime=Time(t_set[wgood],format='isot',scale='utc',location=loc),
                      frame = 'altaz', location = loc)

ra = AltAzcoord.icrs.ra.deg
dec = AltAzcoord.icrs.dec.deg

l = AltAzcoord.galactic.l.deg
b = AltAzcoord.galactic.b.deg

In [ ]:
print(freq_list)

## Map in equatorial coordinates

In [ ]:
######################################################
### Select frequency index to plot:
frq_idx = 0

### Select data product to plot:
data_plot = 10*np.log10(LL_set[wgood,frq_idx])
#data_plot = 10*np.log10(RR_set[wgood,frq_idx])
#data_plot = PI[wgood,frq_idx]

fs=18
Pmin = 72
Pmax = 78
spts = 1
######################################################

print(freq_list[frq_idx])

eq = SkyCoord(ra, dec, frame='icrs', unit=u.deg)

fig = plt.figure(figsize=(20,10))
plt.subplot(111, projection='aitoff')
plt.grid(True)
plt.scatter(-eq.ra.wrap_at('180d').radian, eq.dec.radian,s=spts,c=data_plot,vmin=Pmin, vmax=Pmax,alpha=0.4)
positions = (-150*np.pi/180.,-120*np.pi/180.,-90*np.pi/180.,-60*np.pi/180.,-30*np.pi/180.,0,
             30*np.pi/180.,60*np.pi/180.,90*np.pi/180.,120*np.pi/180.,150*np.pi/180.)
labels = ("10","8","6","4","2","0","22","20","18","16","14")
plt.xticks(positions, labels,fontsize=fs)
plt.yticks(fontsize=fs)
plt.ylabel('Declination (degrees)',fontsize=fs)
plt.xlabel('RA (hours)',fontsize=fs)


## Map in Galactic coordinates

In [ ]:
######################################################
### Select frequency index to plot:
frq_idx = 1

### Select data product to plot:
#data_plot = 10*np.log10(LL_set[wgood,frq_idx])
#data_plot = 10*np.log10(RR_set[wgood,frq_idx])
data_plot = PI[wgood,frq_idx]

fs=18
Pmin = 0
Pmax = 2e5
spts = 1
######################################################

gal = SkyCoord(l, b, frame='galactic', unit=u.deg)

fig = plt.figure(figsize=(20,10))
plt.subplot(111, projection='aitoff')
plt.grid(True)
plt.scatter(-gal.l.wrap_at('180d').radian, gal.b.radian,s=spts,c=data_plot,vmin=Pmin,vmax=Pmax,alpha=0.4)
positions = (-150*np.pi/180.,-120*np.pi/180.,-90*np.pi/180.,-60*np.pi/180.,-30*np.pi/180.,0,
             30*np.pi/180.,60*np.pi/180.,90*np.pi/180.,120*np.pi/180.,150*np.pi/180.)
labels = ("150","120","90","60","30","0","330","300","270","240","210")
plt.xticks(positions, labels,fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlabel('Galactic Longitude (degrees)',fontsize=fs)
plt.ylabel('Galactic Latitude (degrees)',fontsize=fs)

#plt.tight_layout()
#plt.savefig('../DVA2/PLOTS/survey_phase1_47days_800_gal_PI.jpg')